# **IMPORT**

## Import libraries

In [139]:
import pandas as pd
import math

## Load dataset

In [140]:
dataset_path = f'./Datasets/dataset.csv'

with open(dataset_path, 'rt') as f:
    df = pd.read_csv(f)

# **FILTER FUNCTION**

## Define the filter function

[1] "Movies included in the array are those whose even one attribute value matches with the input value of the user."

In [141]:
def Filter(movies_df, minRating=0, maxRating=10, minYear=0, maxYear=math.inf, genres=[], directors=[], authors=[]):

    # Take the movies which are in the range of the rating
    rating_df = movies_df[movies_df['averageRating'] >= minRating]
    rating_df = rating_df[rating_df['averageRating'] <= maxRating]

    # Take the movies which are in the range of the year
    year_df = movies_df[movies_df['startYear'] >= minYear]
    year_df = year_df[year_df['startYear'] <= maxYear]

    # Take the movies from the movies_df such that genres list is contained in the list of comma-separated values in movies_df['genres']
    if genres:
        genres_df = movies_df[movies_df['genres'].apply(lambda x: all(g in x for g in genres))]
    else:
        genres_df = movies_df

    # Take the movies from the movies_df such that directors list is contained in the list of comma-separated values in movies_df['directors']
    if directors:
        directors_df = movies_df[movies_df['directors'].apply(lambda x: all(d in x for d in directors))]
    else:
        directors_df = movies_df

    # Take the movies from the movies_df such that authors list is contained in the list of comma-separated values in movies_df['authors']
    if authors:
        authors_df = movies_df[movies_df['authors'].apply(lambda x: all(a in x for a in authors))]
    else:
        authors_df = movies_df

    # Take the union of all the dataframes avoiding duplicates
    filtered_df = pd.concat([rating_df, year_df, genres_df, directors_df, authors_df]).drop_duplicates() # [1]

    return filtered_df

# **RECOMMENDER FUNCTION**

## Define the recommender function

[1] "In our research we have also found that generally a user prefer a list with five movies so we assume K equal to be 4 so that an average every K has five movies, where K is the number of cluster to be formed."

[2] "For each cluster k1, k2 , k3, k4 we assume initial centroid c1, c2, c3, c4 which corresponds to the first, sixth, eleventh, and sixteenth movie in the movie array."

[3] "The distance measure we have used to calculate the distance between data points and centroid is the Euclidean Distance."

[A] One-hot encoding for categorical variables

In [142]:
def MovieREC(movies_df, n_clusters=4): # [1]

    #  ---- Filtering top 20 movies with highest average rating ---- #
 
    # Return an empty list if the input list is empty
    if len(movies_df) == 0:
        return list()
    
    # Resize the number of clusters based on the number of movies in the input list if there are less than 20 movies
    # Otherwise keep the 20 movies with highest average rating, giving priority by number of ratings
    if len(movies_df) < 20:
        n_clusters = math.ceil(n_clusters * (len(movies_df)/20)) 
    else:
        movies_df = movies_df.sort_values(by=['averageRating', 'numVotes'], ascending=False).head(20)

    # Drop column num_ratings
    movies_df = movies_df.drop(columns=['numVotes'])

    # ---- K-Means Clustering Initialization ---- #

    # Create a dictionary which assigns to each column index in movie_df their name (averageRating, startYear, genres, directors, authors)
    num_of_columns = 5
    col_names = {i: movies_df.columns[i+1] for i in range(num_of_columns)}

    # Now let us create a matrix with num_clusers rows and one column for each column in (averageRating, startYear, genres, directors, authors)
    # Pick evenly spaced n_clusters indeces from the list of movies, starting from the index 0, up to len-1
    centroids_init_indexes = [math.floor(i * len(movies_df)/n_clusters)+1 for i in range(n_clusters)] # [2]
    # Store for each centroid (movie) the values of its corresponding columns in the movies_df for averageRating and startYear
    centroids = [[movies_df.iloc[centroids_init_indexes[j]][col_names[i]] for i in range(2)] for j in range(n_clusters)]
    # And append to each centroid (movie) three dictionaries which assign 1 to the values in the corresponding list, for genres, directors and authors
    for i in range(n_clusters):
        for j in range(2, num_of_columns):
            centroids[i].append( {x: 1 for x in movies_df.iloc[centroids_init_indexes[i]][col_names[j]].split(',')} ) # [A]

    # Create a list of clusters, where each cluster is the list of indexes of the movies in the movies_df
    clusters = [[] for i in range(n_clusters)]
        
    # Initialize to True a "changing" flag, which will be set to False when the centroids stop changing
    changing = True

    # ---- K-Means Algorithm ---- #

    # While the centroids keep changing...
    while changing:

        # Store a copy of the previous clusters
        prev_clusters = clusters.copy()

        # Empty the list of clusters
        clusters = [[] for i in range(n_clusters)]

        # Set the changing flag to False
        changing = False

        # Now, for each movie in the movies_df compute its cluster
        for i in range(len(movies_df)):

            # Save a tuple (math.inf, -1) to store the minimum distance and the index of the closest centroid
            min_distance = (math.inf, -1)

            # And compute the closest centroid for the movie i
            for c_idx in range(n_clusters):

                # Init the one-dimension distances list
                tempDistances = []

                # Compute the 1D distances for the averageRating and startYear columns
                for j in range(2):
                    tempDistances.append(movies_df.iloc[i][col_names[j]] - centroids[c_idx][j])

                # Compute the 1D one-hot tempDistance for genres, directors and authors as well
                for j in range(2, num_of_columns):

                    # By summing 1 for each value in movies_df[col_names[j]][i] not in centroids[c_idx][j].keys()
                    tempBoth = sum([1 for x in movies_df.iloc[i][col_names[j]] if x not in centroids[c_idx][j].keys()])
                    # Summing (1 - centroids[c_idx][j][x]) for each x both in movies_df[col_names[j]][i] and in centroids[c_idx][j].keys()
                    tempMovie = sum([1 - centroids[c_idx][j][x] for x in movies_df.iloc[i][col_names[j]] if x in centroids[c_idx][j].keys()])
                    # Summing centroids[c_idx][j][x] for each x in centroids[c_idx][j].keys() and not in movies_df[col_names[j]][i]
                    tempCentroid = sum([centroids[c_idx][j][x] for x in centroids[c_idx][j].keys() if x not in movies_df.iloc[i][col_names[j]]])

                    # And then appending the overall sum to the 1D distances list
                    tempDistances.append(tempBoth + tempMovie + tempCentroid)

                # Compute the Euclidean distance between the movie i and the centroid c_idx
                distance = math.sqrt(sum([x**2 for x in tempDistances])) # [3]
                                    
                # If the distance is less than the minimum distance, update the minimum distance and the index of the closest centroid
                if distance < min_distance[0]:
                    min_distance = (distance, c_idx)

            # To finally add the movie i to the cluster of the closest centroid
            clusters[min_distance[1]].append(i)

        # Now compute the new centroids as the average of the movies in each cluster
        for i in range(n_clusters):

            # Compute the average of the averageRating and startYear columns (easy)
            centroids[i] = [sum([movies_df.iloc[j][col_names[k]] for j in clusters[i]])/len(clusters[i]) for k in range(2)]

            # And append to the new centroids three dictionaries which assign the avg for each value in genres, directors and authors with respect to the movies in the cluster
            # Remember we started using one-hot encoding, so the average will be a number between 0 and 1 for each value of each column
            for i in range(n_clusters):
                for j in range(2, num_of_columns):
                    # Compute occurrencies of values in the current column for all the movies in the current cluster
                    centroids[i].append({})
                    for m in clusters[i]:
                        for v in movies_df.iloc[m][col_names[j]].split(','):
                            if v in centroids[i][j].keys():
                                centroids[i][j][v] += 1
                            else:
                                centroids[i][j][v] = 1
                    # And make them an "average"
                    centroids[i][j] = {key: value / len(clusters[i]) for key, value in centroids[i][j].items()}    

        # Now set the changing flag to True if CLUSTERS have changed
        for i in range(n_clusters):
            for j in range(num_of_columns):
                if set(clusters[i]) != set(prev_clusters[i]):
                    changing = True
                    break

        # If they did not change, K-Means has converged and we will stop

    # ---- Pick best cluster ---- #
            
    clusters_dict = {}

    # Compute the weighted average movie rating for each cluster
    # Each cluster contains at least one movie with non-zero weight, so we can safely compute the weighted average
    for idx, cl in enumerate(clusters):
        weights_sum = 0
        weighted_sum = 0
        for i in cl:
            weights_sum += movies_df.iloc[i]['weight']
            weighted_sum += movies_df.iloc[i]['averageRating'] * movies_df.iloc[i]['weight']
        clusters_dict[idx] = weighted_sum/weights_sum

    # Return the cluster with the highest weighted average movie rating
    return clusters[max(clusters_dict, key=clusters_dict.get)] # max() returns the first key with the highest value


# **EXECUTION**

## Find recommendations with no filtering

In [144]:
df.iloc[MovieREC(Filter(df))]

,tconst,averageRating,numVotes,startYear,genres,directors,writers,weight
11,tt0041005,6.0,41,1949,"Drama,History","nm0146769,nm2577273,nm1802165",nm2436797,6.006435
12,tt0041007,6.2,56,1949,Drama,"nm0884037,nm0230529,nm1037509,nm0773780,nm0723...","nm3008552,nm0096395,nm0096831,nm1709258,nm1412...",6.330640


## Check it is deterministic

In [145]:
df.iloc[MovieREC(Filter(df))]

,tconst,averageRating,numVotes,startYear,genres,directors,writers,weight
11,tt0041005,6.0,41,1949,"Drama,History","nm0146769,nm2577273,nm1802165",nm2436797,6.006435
12,tt0041007,6.2,56,1949,Drama,"nm0884037,nm0230529,nm1037509,nm0773780,nm0723...","nm3008552,nm0096395,nm0096831,nm1709258,nm1412...",6.330640


## Try by filtering out the preceiding result

In [147]:
df.iloc[MovieREC(Filter(df, minRating=8, maxRating=10))]

,tconst,averageRating,numVotes,startYear,genres,directors,writers,weight
11,tt0041005,6.0,41,1949,"Drama,History","nm0146769,nm2577273,nm1802165",nm2436797,6.006435
12,tt0041007,6.2,56,1949,Drama,"nm0884037,nm0230529,nm1037509,nm0773780,nm0723...","nm3008552,nm0096395,nm0096831,nm1709258,nm1412...",6.330640
